### **Annotation by RGB**

In [ ]:
import cv2
import numpy as np
import os

def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return None

    # Convert to HSV space
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Setting HSV range
    hue_center = 120
    saturation_center = 85
    value_center = 195

    # Colour range
    hue_range = 10
    saturation_range = 40
    value_range = 60

    lower_purple = np.array([hue_center - hue_range, saturation_center - saturation_range, value_center - value_range])
    upper_purple = np.array([hue_center + hue_range, saturation_center + saturation_range, value_center + value_range])

    # Creat Mask
    mask = cv2.inRange(image_hsv, lower_purple, upper_purple)

    # Extract Purple area
    result = cv2.bitwise_and(image, image, mask=mask)
    return result

def batch_process(input_folder, output_folder):
    # Output
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(input_folder, filename)
            result = process_image(image_path)
            if result is not None:
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, result)
                print(f"Processed and saved: {output_path}")
            else:
                print(f"Processing failed for: {image_path}")

# apply Phase map here for extraction
input_folder = '/Mineral/Phase crop'
output_folder = '/Mineral/cpx'

batch_process(input_folder, output_folder)

### **Generate the mask and apply for target maps**

In [ ]:
import cv2
import numpy as np
import os

def process_image(bse_path, phase_path, output_path):
    phase_image = cv2.imread(phase_path)
    bse_image = cv2.imread(bse_path)

    # generate the mask
    gray_mask = cv2.cvtColor(phase_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)

    # Extract CPX from BSE map
    extracted_regions = cv2.bitwise_and(bse_image, bse_image, mask=binary_mask)

    cv2.imwrite(output_path, extracted_regions)
    print(f"process and save: {output_path}")

def batch_process(bse_folder, phase_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(bse_folder):
        if filename.lower().endswith(".png"):
            bse_path = os.path.join(bse_folder, filename)

            modified_filename = filename.replace("BSE", "_Panorama_Phases+BSE_mask") #replace this when processing different image files
            phase_path = os.path.join(phase_folder, modified_filename)


            output_path = os.path.join(output_folder, filename)

            print(f"Processing BSE images: {bse_path}")
            print(f"Processing Phase map: {phase_path}")

            if not os.path.exists(phase_path):
                print(f"cannot load image: {phase_path}")
                continue

            process_image(bse_path, phase_path, output_path)

bse_folder = '/Mineral/BSE'
phase_folder = '/Mineral/MASK'
output_folder = '/Mineral/BSE-A'
batch_process(bse_folder, phase_folder, output_folder)


### **Canny algorithm segmentation files**

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd


root_dir = '/Mineral/BSE'
save_dir = '/Mineral/BSE-seg'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# recoed the segmentation
segment_data = []

for filename in glob.iglob(root_dir + '**/*.png', recursive=True):
    img = cv2.imread(filename, cv2.IMREAD_COLOR)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 20, 255, 1)  # Canny edge detection


    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #match the filename
    base_filename = os.path.basename(filename)
    image_subdir = os.path.join(save_dir, os.path.splitext(base_filename)[0])
    if not os.path.exists(image_subdir):
        os.makedirs(image_subdir)


    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        if w > 50 and h > 50:  # size of capturing
            ROI = img[y:y+h, x:x+w]
            output_filename = f"{base_filename}_segment_{i}.png"  # creat filenames
            output_path = os.path.join(image_subdir, output_filename)
            cv2.imwrite(output_path, ROI)

            segment_data.append({
                'Filename': output_filename,
                'Segment Index': i,
                'x': x,
                'y': y,
                'width': w,
                'height': h
            })


segmentation_info = pd.DataFrame(segment_data)

excel_filename = os.path.join('/Mineral/Test set', 'TEST_segmentation.xlsx')
segmentation_info.to_excel(excel_filename, index=False)

print("Segmentation completed and images are saved. Segmentation details are recorded in the Excel file.")


Segmentation completed and images are saved. Segmentation details are recorded in the Excel file.


### **Load the segmentation record for further segmentation on other maps**

In [ ]:
import pandas as pd
import os
csv_file = '/Mineral/segmentation_record.csv'
segmentation_record = pd.read_csv(csv_file)

segmentation_record_sorted = segmentation_record.sort_values(by=['Original Filename'])

segmentation_record_sorted['Key'] = segmentation_record_sorted['Original Filename'].apply(lambda x: os.path.basename(x).split('_')[0])

segmentation_record_sorted.head()


,Original Filename,Segmented Filename,X,Y,Width,Height,Key
256,/content/drive/MyDrive/Mineral Sample/CPX ENHA...,/content/drive/MyDrive/Mineral Sample/CPX seg/...,1396,222,133,67,02NE05A - Modal analysis #1
259,/content/drive/MyDrive/Mineral Sample/CPX ENHA...,/content/drive/MyDrive/Mineral Sample/CPX seg/...,3213,72,83,86,02NE05A - Modal analysis #1
257,/content/drive/MyDrive/Mineral Sample/CPX ENHA...,/content/drive/MyDrive/Mineral Sample/CPX seg/...,3282,170,286,173,02NE05A - Modal analysis #1
255,/content/drive/MyDrive/Mineral Sample/CPX ENHA...,/content/drive/MyDrive/Mineral Sample/CPX seg/...,1555,237,114,51,02NE05A - Modal analysis #1
254,/content/drive/MyDrive/Mineral Sample/CPX ENHA...,/content/drive/MyDrive/Mineral Sample/CPX seg/...,1291,293,282,384,02NE05A - Modal analysis #1


In [ ]:
import cv2
import numpy as np

save_dir = '/Mineral/BSE' # replace the document here to segment different SEM maps
os.makedirs(save_dir, exist_ok=True)

for index, row in segmentation_record_sorted.iterrows():
    key = row['Key']
    matches = [f for f, p_key in zip(phase_map_files, phase_map_keys) if key == p_key]

    if matches:
        phase_map_file = matches[0]
        img_path = os.path.join(new_images_dir, phase_map_file)
        img = cv2.imread(img_path)

        x, y, w, h = row['X'], row['Y'], row['Width'], row['Height']
        segmented_img = img[y:y+h, x:x+w]

        save_path = os.path.join(save_dir, os.path.basename(row['Segmented Filename']))
        cv2.imwrite(save_path, segmented_img)

        print(f"Saved segmented image to {save_path}")
